# NetCDF read/write
This notebook is focused on the [netCDF](https://www.unidata.ucar.edu/software/netcdf/) and how to read/write data from/in this format.    
The first module to work with netCDF was https://github.com/JuliaGeo/NetCDF.jl, but recently [@abarth](https://github.com/Alexander-Barth) developed another module: [NCDatasets](https://github.com/Alexander-Barth/NCDatasets.jl). We will use the later in this notebook.

In [1]:
using NCDatasets
Pkg.status("NCDatasets")

 - NCDatasets                    0.0.10


## Read a netCDF
We will use the GEBCO bathymetry as an example.

In [3]:
ncfile = "gebco_30sec_16.nc"
if !isfile(ncfile)
    download("https://b2drop.eudat.eu/s/o0vinoQutAC7eb0/download", ncfile)
else
    info("Bathymetry file already downloaded")
end

INFO: Bathymetry file already downloaded


First we open the file for readind (**"r"** option).    

In [7]:
ds = Dataset(ncfile, "r");
@show ds;

ds = Dataset: gebco_30sec_16.nc
Group: /

Dimensions
   lat = 1352
   lon = 2700

Variables
  lat   (1352)
    Datatype:    Float64
    Dimensions:  lat
    Attributes:
     long_name            = Latitude
     standard_name        = latitude
     units                = degrees_north

  lon   (2700)
    Datatype:    Float64
    Dimensions:  lon
    Attributes:
     long_name            = Longitude
     standard_name        = longitude
     units                = degrees_east

  bat   (2700 × 1352)
    Datatype:    Float32
    Dimensions:  lon × lat
    Attributes:
     long_name            = elevation above sea level
     standard_name        = height
     units                = meters

Global attributes
  title                = GEBCO



We now know the variable name and attributes, as well as the dimensions.    
Let's read the longitude, latitude and bathymetry.

In [9]:
lon = ds["lon"];
lat = ds["lat"];
bathymetry = ds["bat"];

In [ ]:
Note that with the previous commands, we load the variable values and attributes.

In [15]:
@show typeof(lon);
@show lon.attrib;

typeof(lon) = NCDatasets.CFVariable{Float64,Float64,1}
lon.attrib =   long_name            = Longitude
  standard_name        = longitude
  units                = degrees_east



If we want to access the values only, we can use similar commands but with the `[:]` at the end:

In [16]:
lon = ds["lon"][:];
lat = ds["lat"][:];
bathymetry = ds["bat"][:];

In [ ]:
which yield data arrays

In [18]:
@show typeof(lon);

typeof(lon) = DataArrays.DataArray{Float64,1}


## Write a netCDF
### Time series
Let's start with a very simple example: a temperature time series (1-dimensional dataset), corresponding to the mean temperature in Uccle, Belgium (source: [IRM](https://www.meteo.be/meteo/view/fr/360955-Normales+mensuelles.html#ppt_5238240)).

In [49]:
temperatureUccle = [3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18., 14.9, 11.1, 6.8, 3.9]

12-element Array{Float64,1}:
  3.3
  3.7
  6.8
  9.8
 13.6
 16.2
 18.4
 18.0
 14.9
 11.1
  6.8
  3.9

First, let's create the file (clobber mode indcated by "c"):

In [53]:
ds = Dataset("temperature_series.nc", "c");

Then we create a time dimension. Its size should correspond to that of our temperature vector.

In [54]:
defDim(ds, "time", length(temperatureUccle))

0

Now we can also create the *temperature* variable, which has *time* as dimension.

In [55]:
temperature = defVar(ds, "temperature", Float64, ("time",));

We will this variable with the values previously defined:

In [56]:
temperature[:] = temperatureUccle;

and don't forget to close the `ds`:

In [57]:
close(ds)

If you have [`ncdump`] installed on your machine, you can easily check the file content.     
Otherwise we'll show another way to do it.

In [59]:
run(`ncdump temperature_series.nc`)

netcdf temperature_series {
dimensions:
	time = 12 ;
variables:
	double temperature(time) ;
data:

 temperature = 3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18, 14.9, 11.1, 6.8, 3.9 ;
}


In [ ]:
## 2D field
The procedure is similar to the previous example, except that we will work with longitude and latitude coordinates.